# PE Halo

This notebook will exercise your learning of Serialbox and porting from Fortran to Python.



In [14]:
%%writefile edge_pressure.F90

subroutine compute_edge_pressure(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
    integer, intent(in) :: is, ie, js, je, isd, ied, jsd, jed, npz
    double precision, intent(in) :: ptop
    double precision, intent(in), dimension(isd:ied,jsd:jed,npz):: delp
    double precision, intent(inout), dimension(is-1:ie+1,npz+1,js-1:je+1):: pe
    integer:: i,j,k

    do j=js,je
     pe(is-1,1,j) = ptop
     pe(ie+1,1,j) = ptop
     do k=1,npz
        pe(is-1,k+1,j) = pe(is-1,k,j) + delp(is-1,j,k)
        pe(ie+1,k+1,j) = pe(ie+1,k,j) + delp(ie+1,j,k)
     enddo
    enddo
    
    do i=is-1,ie+1
     pe(i,1,js-1) = ptop
     pe(i,1,je+1) = ptop
     do k=1,npz
        pe(i,k+1,js-1) = pe(i,k,js-1) + delp(i,js-1,k)
        pe(i,k+1,je+1) = pe(i,k,je+1) + delp(i,je+1,k)
     enddo
  enddo
end subroutine

program edge_pressure

  implicit none
    
  integer :: is, ie, js, je, isd, ied, jsd, jed, npz
  double precision :: ptop
  double precision, allocatable, dimension(:,:,:) :: delp, pe
  !$ser init directory='./data_pe' prefix='port' unique_id=.true.
  !$ser mode write
  !$ser on
  is = 1
  ie = 48
  js = 1
  je = 48
  isd = -2
  ied = 51
  jsd = -2
  jed = 51
  npz = 79
  ptop = 300.0
  allocate(delp(isd:ied, jsd:jed, npz))
  allocate(pe(is-1:ie+1,npz+1,js-1:je+1))
  !$ser savepoint 'edge_pressure-in'
  !$ser data ptop=ptop pe=pe delp=delp is_=is ie_=ie js_=js je_=je isd=isd ied=ied jsd=jsd jed=jed npz=npz
  call compute_edge_pressure(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
  !$ser savepoint 'edge_pressure-out'
  !$ser data pe=pe
  !$ser cleanup
end program

Overwriting edge_pressure.F90


In [15]:
%%bash
python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_edge_pressure.F90 edge_pressure.F90
gfortran -O3 -cpp -DSERIALIZE \
    -o serialize_edge_pressure s_edge_pressure.F90 \
    -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
    -lpthread -lstdc++ -lstdc++fs
rm -rf ./data_pe
./serialize_edge_pressure
ls -lh data_pe

Processing file edge_pressure.F90
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
total 4.9M
-rw-r--r-- 1 rgeorge noaa-hpc 1.7K Nov 11 18:53 ArchiveMetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc  14K Nov 11 18:53 MetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc 1.8M Nov 11 18:53 port_delp.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_ie_.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_ied.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_is_.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_isd.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_je_.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_jed.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_js_.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_jsd.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 11 18:53 port_npz.dat
-rw-r--r-- 1 rgeorge noaa-hpc 3.1M Nov 11 18:53 port_pe.dat
-rw-r--r-- 1 rgeorge noaa-hpc

### Python Solution A
Using array size that is the max shape of all data arrays, matching indices

In [20]:
#!/usr/bin/env python3

import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser

field = gtscript.Field[np.float64]
backend="numpy"
origin=(0, 0, 0)
fortran2python_offset = 2

@gtscript.stencil(backend=backend)
def edge_pressure(pe: field, delp: field, ptop: float):
    with computation(FORWARD):
        with interval(0, 1):
            pe[0, 0, 0] = ptop
        with interval(1, None):
            pe[0, 0, 0] = pe[0, 0, -1] + delp[0, 0, -1]

def compute(pe, delp, ptop, is_, ie_, js_, je_, npz):
    istart = is_ - 1
    edge_domain_x = (1, je_ - js_ + 1, npz + 1)
    edge_pressure(pe, delp, ptop, origin=(istart, js_, 0), domain=edge_domain_x)
    edge_pressure(pe, delp, ptop, origin=(ie_ + 1, js_, 0), domain=edge_domain_x)
    edge_domain_y = (ie_ - is_ + 3, 1, npz + 1)
    edge_pressure(pe, delp, ptop, origin=(istart, js_ - 1, 0), domain=edge_domain_y)
    edge_pressure(pe, delp, ptop, origin=(istart, je_ + 1, 0), domain=edge_domain_y)

def serialized_index(serializer, savepoint, name):
    return int(serializer.read(name,  savepoint[0])[0]) + fortran2python_offset

serializer = ser.Serializer(ser.OpenModeKind.Read,'./data_pe', 'port')

sp_in = serializer.get_savepoint('edge_pressure-in')

pe = serializer.read('pe',  sp_in[0])
delp = serializer.read('delp',  sp_in[0])
ptop = serializer.read('ptop',  sp_in[0])[0]

for index_name in ['is_', 'ie_', 'js_', 'je_', 'isd', 'ied', 'jsd', 'jed']:
    locals()[index_name] = serialized_index(serializer, sp_in, index_name)
npz = int(serializer.read('npz',  sp_in[0])[0]) 

# reshape pe it be i, j, k
pe = np.moveaxis(pe, 1, 2)
# pick a storage size that encompasses both delp and pe
storage_shape = (ied - isd + 1, jed - isd + 1, npz +1)
# make pe gt storage
pe_overlap = np.empty(storage_shape)
pe_overlap[is_ - 1:ie_ + 2, js_ - 1: je_ + 2, :] = pe
pe =  gt_storage.from_array(data=pe_overlap, backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape)
# make delp storage
delp_overlap = np.empty(storage_shape)
delp_overlap[:, :, 0:npz] = delp
delp = gt_storage.from_array(data=delp_overlap,  backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape,)
# call the computation
compute(pe, delp, ptop, is_, ie_, js_, je_, npz)
# Get regression data
sp_out = serializer.get_savepoint('edge_pressure-out')
pe_ref = serializer.read('pe',  sp_out[0])
# reformat computed value back
pe = pe[is_ - 1:ie_ + 2, js_ - 1: je_ + 2, :] 
pe =np.moveaxis(pe, 2, 1)
# compare answers
try:
    assert np.array_equal(pe_ref, pe), "pe does not match!"
except AssertionError as msg:
    print(msg)
    
print("Finished running comparison tests!")

Finished running comparison tests!


### Python Solution B
Using array size that is the minimum size required for this computation

In [21]:
#!/usr/bin/env python3

import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser

field = gtscript.Field[np.float64]
backend="numpy"
origin=(0, 0, 0)
@gtscript.stencil(backend=backend)
def edge_pressure(pe: field, delp: field, ptop: float):
    with computation(FORWARD):
        with interval(0, 1):
            pe[0, 0, 0] = ptop
        with interval(1, None):
            pe[0, 0, 0] = pe[0, 0, -1] + delp[0, 0, -1]

def compute(pe, delp, ptop, ie_, je_, npz):
    edge_domain_x = (1, je_, npz + 1)
    edge_pressure(pe, delp, ptop, origin=(0, 1, 0), domain=edge_domain_x)
    edge_pressure(pe, delp, ptop, origin=(ie_ + 1, 1, 0), domain=edge_domain_x)
    edge_domain_y = (ie_ + 2, 1, npz + 1)
    edge_pressure(pe, delp, ptop, origin=(0, 0, 0), domain=edge_domain_y)
    edge_pressure(pe, delp, ptop, origin=(0, je_ + 1, 0), domain=edge_domain_y)

def serialized_index(serializer, savepoint, name):
    return int(serializer.read(name,  savepoint[0])[0]) + fortran2python_offset

serializer = ser.Serializer(ser.OpenModeKind.Read,'./data_pe', 'port')

sp_in = serializer.get_savepoint('edge_pressure-in')

pe = serializer.read('pe',  sp_in[0])
delp = serializer.read('delp',  sp_in[0])
ptop = serializer.read('ptop',  sp_in[0])[0]
fortran2python_offset = 2
for index_name in ['is_', 'ie_', 'js_', 'je_', 'isd', 'ied', 'jsd', 'jed']:
    locals()[index_name] = serialized_index(serializer, sp_in, index_name)

# reshape pe it be i, j, k
pe = np.moveaxis(pe, 1, 2)
# Use the smallest storage size necessary
storage_shape=(ie_ + 1 - (is_ - 1) + 1, je_ + 1 - (js_ - 1) + 1, npz+1)
# make pe gt storage
pe =  gt_storage.from_array(data=pe, backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape)
# make delp storage, only the subset used in this computation
delp_overlap = np.empty(storage_shape)
delp_overlap[:, :, 0:npz] = delp[is_ - 1: ie_ + 2, js_ - 1: je_ + 2,:]
delp = gt_storage.from_array(data=delp_overlap,  backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape,)
compute(pe, delp, ptop, ie_ - is_ + 1, je_ - js_ + 1, npz)
# Get regression data
sp_out = serializer.get_savepoint('edge_pressure-out')
pe_ref = serializer.read('pe',  sp_out[0])
# reformat computed value back 
pe =  np.moveaxis(pe, 2, 1)
# compare answers
try:
    assert np.array_equal(pe_ref, pe), "pe does not match!"
except AssertionError as msg:
    print(msg)
    
print("Finished running comparison tests!")

Finished running comparison tests!
